In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre_10sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')
#train_afds

In [3]:
train_lre_10sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.314181,0.671920,-0.188068,-0.910752,1.050361,1.966829,0.292740,1.084547,1.287480,1.745248,...,-0.564911,-1.854116,-3.185948,-2.177057,0.301166,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.503734,0.247072,0.451463,0.500546,-0.716668,1.337575,-1.027019,0.773720,0.910362,1.728001,...,0.886613,6.766799,-1.040380,-0.751316,-1.487938,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.141677,-0.044947,-0.595182,-0.595953,0.889385,1.440080,-1.208933,0.129059,-0.930121,0.401362,...,-2.836253,-0.323334,0.055760,0.481039,0.395826,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.708722,-0.663542,0.031733,0.358099,0.594610,1.023025,-1.450633,0.252159,-0.381832,0.087693,...,1.384728,0.426863,0.482178,1.655650,-1.029032,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.873292,-0.272493,-0.305948,-0.250047,2.093700,1.741634,-0.750633,-0.651771,-0.793287,-0.053527,...,0.188493,-0.888710,1.792536,-0.628455,1.912622,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [4]:
X_train_10sec = train_lre_10sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [5]:
#train_lre = train_lre.iloc[1000:2000]

In [6]:
train_lre_30sec = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_X_train_04Jan_labels_ids_combined_extraction.csv')


In [7]:
train_lre_30sec.head()

,0,1,2,3,4,5,6,7,8,9,...,495,496,497,498,499,ids,year,data,lang,lang_id
0,-0.639420,0.345684,-0.517645,-0.737002,1.313001,1.655732,0.615168,0.799339,1.648419,1.314986,...,-1.322407,-0.717348,-3.843951,-1.471274,0.945421,zkllk,LDC2017E22,data,eng-usg,zkllk
1,0.113610,0.738034,0.584856,-0.248521,0.256616,1.060159,-0.416211,0.133670,0.188327,0.805241,...,1.965741,3.525555,0.006925,1.676099,-1.116371,lid05e1_lid00562,LDC2017E22,data,eng-usg,lid05e1_lid00562
2,2.061305,-0.140605,-0.627104,-0.682908,1.337613,1.261543,-0.651291,0.307040,-0.980717,0.335530,...,-3.018432,-1.841361,0.814595,-0.200731,1.180795,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
3,1.958049,-0.608315,-0.113532,0.167272,0.911100,1.116094,-0.847214,-0.041357,-0.542335,0.199105,...,-0.548767,-1.254447,1.644647,0.555434,1.159140,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a
4,1.842436,-0.417598,-0.427428,-0.485898,1.204330,1.439188,-1.056642,0.111603,-0.404313,-0.307075,...,0.297026,-1.858708,0.209784,0.583939,1.398789,fla_0240-a,LDC2017E22,data,ara-apc,fla_0240-a


In [8]:
X_train_30sec = train_lre_30sec.drop(['ids','year','data','lang','lang_id'],axis=1)

In [9]:
y_30sec_labels = train_lre_30sec["lang"]

In [10]:
y_30sec_labels.head()

0    eng-usg
1    eng-usg
2    ara-apc
3    ara-apc
4    ara-apc
Name: lang, dtype: object

In [11]:
le = preprocessing.LabelEncoder()
le.fit(y_30sec_labels)

LabelEncoder()

In [12]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [13]:
y_30sec_labels=le.transform(y_30sec_labels)

In [14]:
y_30sec_labels.shape

(114276,)

In [15]:
nb_classes = 14
# convert class vectors to binary class matrices
Y_train_labels = np_utils.to_categorical(y_30sec_labels, nb_classes)

In [16]:
Y_train_labels.shape

(114276, 14)

In [17]:
X_train_10sec = X_train_10sec.values
X_train_30sec = X_train_30sec.values

In [18]:
X_train_10sec = X_train_10sec.astype('float32')
#X_test = X_test.astype('float32')
X_train_30sec = X_train_30sec.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train_10sec shape:', X_train_10sec.shape)
print(X_train_10sec.shape[0], 'train 10sec')
#print(X_test.shape[0], 'test samples')
print(X_train_30sec.shape[0], ' train 30sec')


X_train_10sec shape: (114276, 500)
114276 train 10sec
114276  train 30sec


In [19]:
#Shuffle the Dataset
X_train_10sec_F,  X_train_30sec_F, y_30sec_labels, Y_train_labels = shuffle(X_train_10sec, X_train_30sec,y_30sec_labels, Y_train_labels, random_state=0)

In [20]:
import torch.nn as nn

In [21]:
class FC_Network(torch.nn.Module):

    def __init__(self,):
        super(FC_Network, self).__init__()

        D_in = D_out = 500        # param['patch_length'] * param['n_channels'] * (param['n_fft'] / 2 + 1)
        num_nodes_fnn = 512
        self.layer_1 = torch.nn.Linear(D_in, num_nodes_fnn)
        self.bn_1 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_2 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_2 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_3 = torch.nn.Linear(num_nodes_fnn, num_nodes_fnn)
        self.bn_3 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.layer_4 = torch.nn.Linear(num_nodes_fnn,num_nodes_fnn)
        self.bn_4 = torch.nn.BatchNorm1d(num_nodes_fnn)
        self.output_layer = torch.nn.Linear(num_nodes_fnn, D_out)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        out = x
        #out = x.view(x.size(0), -1)
        out = self.bn_1(self.relu(self.layer_1(out)))
        out = self.bn_2(self.relu(self.layer_2(out)))
        out = self.bn_3(self.relu(self.layer_3(out)))
        out = self.bn_4(self.relu(self.layer_4(out)))
        out = self.output_layer(out)

        #out = (self.relu(self.layer_1(out)))
        #out = (self.relu(self.layer_2(out)))
        #out = (self.relu(self.layer_3(out)))
        #out = (self.relu(self.layer_4(out)))
        #out = (self.output_layer(out))
        #out = out.view(x.size())
        return out

In [22]:
G = FC_Network()

In [23]:
G.cuda()

FC_Network(
  (layer_1): Linear(in_features=500, out_features=512)
  (bn_1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_2): Linear(in_features=512, out_features=512)
  (bn_2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_3): Linear(in_features=512, out_features=512)
  (bn_3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (layer_4): Linear(in_features=512, out_features=512)
  (bn_4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True)
  (output_layer): Linear(in_features=512, out_features=500)
  (relu): ReLU()
)

In [24]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [25]:
# Loss function
criterion = torch.nn.MSELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
#D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [26]:
len(X_train_10sec)

114276

In [27]:
len(X_train_30sec)

114276

In [28]:

mb_size = 256
batch_size = mb_size
# Start training
for epoch in range(100):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train_10sec_F) :
        

        start= it
        end= it + batch_size


        #z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train_10sec_F[start:end]

        c = X_train_30sec_F[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))
        X = X.cuda()
        c = c.cuda()
        # Dicriminator forward-loss-backward-update
        G_sample = G(X)
        #D_real = D(X, c)
        #D_fake = D(G_sample, c)

        #D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        #D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        #D_loss = D_loss_real + D_loss_fake

        #D_loss.backward()
        #D_solver.step()

        # Housekeeping - reset gradient
        #D.zero_grad()

        # Generator forward-loss-backward-update
        #z = Variable(torch.randn(mb_size, Z_dim))
        #G_sample = G(z, c)
        #D_fake = D(G_sample, c)
        G_loss = criterion(G_sample, c)
        #G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()
        
        G_solver.zero_grad()
        # Housekeeping - reset gradient
        #D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        #print('Iter-{}; G_loss: {}'.format(it, G_loss.data.numpy()))



        it+= batch_size
    print('G_loss: {}'.format(G_loss.cpu().data.numpy()))
           

epoch: 0
G_loss: [ 1.39367688]
epoch: 1
G_loss: [ 1.21248233]
epoch: 2
G_loss: [ 1.1203233]
epoch: 3
G_loss: [ 1.06714106]
epoch: 4
G_loss: [ 1.03395903]
epoch: 5
G_loss: [ 1.0128094]
epoch: 6
G_loss: [ 0.99604368]
epoch: 7
G_loss: [ 0.98334318]
epoch: 8
G_loss: [ 0.97485036]
epoch: 9
G_loss: [ 0.96568239]
epoch: 10
G_loss: [ 0.9588238]
epoch: 11
G_loss: [ 0.95325392]
epoch: 12
G_loss: [ 0.947869]
epoch: 13
G_loss: [ 0.94264174]
epoch: 14
G_loss: [ 0.93954772]
epoch: 15
G_loss: [ 0.9359079]
epoch: 16
G_loss: [ 0.93246794]
epoch: 17
G_loss: [ 0.92925042]
epoch: 18
G_loss: [ 0.92690641]
epoch: 19
G_loss: [ 0.92495042]
epoch: 20
G_loss: [ 0.92246634]
epoch: 21
G_loss: [ 0.92010808]
epoch: 22
G_loss: [ 0.91844285]
epoch: 23
G_loss: [ 0.91597706]
epoch: 24
G_loss: [ 0.91398275]
epoch: 25
G_loss: [ 0.91215843]
epoch: 26
G_loss: [ 0.90993088]
epoch: 27
G_loss: [ 0.90840733]
epoch: 28
G_loss: [ 0.90676546]
epoch: 29
G_loss: [ 0.9050501]
epoch: 30
G_loss: [ 0.90351003]
epoch: 31
G_loss: [ 0.901

In [ ]:
X_train_30sec_gen = Variable(torch.from_numpy(X_train_10sec_F))

In [ ]:
G.eval()

In [ ]:
gen_samples = G(X_train_30sec_gen.cuda())

In [ ]:
gen_samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X_gen = gen_samples.cpu().data.numpy()

In [ ]:
type(train_X_gen)

In [ ]:
X_train = pd.DataFrame(X_train_10sec)
#Y_train = pd.DataFrame(Y_train)


In [ ]:
val_lre = pd.read_csv('/home/satishk/GAN_lre/gan_csv/dev_feat_BNF_h5_05Jan_10sec.csv')


In [ ]:
val_lre.head()

In [ ]:
X_val = val_lre.drop(["language_code","uttid","segmentid1","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid1"]

In [ ]:
X_val = X_val.values
#X_eval = X_eval.values

In [ ]:
#X_eval.shape

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

In [ ]:
le.classes_

In [ ]:
y_val_labels = le.transform(y_val)

In [ ]:
y_val_labels[0:25]

In [ ]:
# convert class vectors to binary class matrices
Y_val = np_utils.to_categorical(y_val_labels, nb_classes)
#Y_eval = np_utils.to_categorical(y_eval_labels, nb_classes)

In [ ]:
Y_val

In [ ]:
X_val = X_val.astype('float32')

In [ ]:
X_val_10sec_gen = Variable(torch.from_numpy(X_val))

In [ ]:
gen_samples_val = G(X_val_10sec_gen.cuda())

In [ ]:
gen_samples_val.data.shape

In [ ]:
X_val_gen = gen_samples_val.cpu().data.numpy()

In [ ]:
train_X_gen.shape

In [ ]:
X_val_gen = pd.DataFrame(X_val_gen)
train_X_gen = pd.DataFrame(train_X_gen)
X_val = pd.DataFrame(X_val)
X_train_10sec_F = pd.DataFrame(X_train_10sec_F)
Y_train_labels = pd.DataFrame(Y_train_labels)
y_30sec_labels = pd.DataFrame(y_30sec_labels)
y_val_labels = pd.DataFrame(y_val_labels)
Y_val = pd.DataFrame(Y_val)

In [ ]:
X_val_gen.to_csv('/home/satishk/GAN_lre/gan_csv/X_val_gen.csv',index=False)

train_X_gen.to_csv('/home/satishk/GAN_lre/gan_csv/train_X_gen.csv',index=False)

X_val.to_csv('/home/satishk/GAN_lre/gan_csv/X_val.csv',index=False)

X_train_10sec_F.to_csv('/home/satishk/GAN_lre/gan_csv/X_train_10sec_F.csv',index=False)

Y_train_labels.to_csv('/home/satishk/GAN_lre/gan_csv/Y_train_labels.csv',index=False)

y_30sec_labels.to_csv('/home/satishk/GAN_lre/gan_csv/y_30sec_labels.csv',index=False)

y_val_labels.to_csv('/home/satishk/GAN_lre/gan_csv/y_val_labels.csv',index=False)

Y_val.to_csv('/home/satishk/GAN_lre/gan_csv/Y_val.csv',index=False)


In [ ]:
#from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import accuracy_score

In [ ]:
X_train_10sec_F=pd.read_csv('/home/satishk/GAN_lre/gan_csv/X_train_10sec_F.csv',header=None)

y_30sec_labels=pd.read_csv('/home/satishk/GAN_lre/gan_csv/y_30sec_labels.csv',header=None)

train_X_gen=pd.read_csv('/home/satishk/GAN_lre/gan_csv/train_X_gen.csv',header=None)

X_val_gen=pd.read_csv('/home/satishk/GAN_lre/gan_csv/X_val_gen.csv',header=None)

X_val=pd.read_csv('/home/satishk/GAN_lre/gan_csv/X_val.csv',header=None)

In [ ]:
y_val_labels=pd.read_csv('/home/satishk/GAN_lre/gan_csv/y_val_labels.csv',header=None)

In [ ]:
#Y_val.values
X_val_gen.values
train_X_gen.values
X_val.values
X_train_10sec_F.values

In [ ]:
#Y_train_labels.values

In [ ]:
classif = SVC(C=1, kernel='rbf', gamma='auto', coef0=1, shrinking=True, random_state=0,
                      probability=False, tol=1e-3, cache_size=1e4,)

In [ ]:
classif.fit(X_train_10sec_F, y_30sec_labels)

#classif.classes_

In [ ]:
acc = accuracy_score(y_val_labels, classif.predict(X_val))

acc